# Tarea 5 - Árbol de Desición

***Inteligencia Artificial***

***II Semestre 2024***

***Tecnológico de Costa Rica***

Estudiantes: 
- Esteban Guzmán
- Rolando Mora

In [22]:
import pandas as pd
import numpy as np

## Actividad - Taller

1. Cree una clase nodo con atributos necesarios para un árbol de decisión: feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha

In [23]:
class Nodo:
    # :param feature: Índice de la característica utilizada para la división en este nodo.
    # :param umbral: Valor del umbral utilizado para la división.
    # :param gini: Índice de Gini de este nodo.
    # :param cantidad_muestras: Número de muestras en este nodo.
    # :param valor: Valor de la clase para nodos hoja (puede ser el valor promedio o la clase más común).
    # :param izquierda: Nodo hijo izquierdo.
    # :param derecha: Nodo hijo derecho.
    def __init__(self, feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha):
        self.feature = feature
        self.umbral = umbral
        self.gini = gini
        self.cantidad_muestras = cantidad_muestras
        self.valor = valor
        self.izquierda = izquierda
        self.derecha = derecha

2. Crea una clase que implementa un árbol de decisión, utilice las funciones presentadas en clase, además incluya los siguientes hyperparámetros:
- max_depth: Cantidad máxima de variables que se pueden explorar
- min_split_samples: Cantidad mínima de muestras que deberá tener un nodo para poder ser dividido
- criterio: función que se utilizará para calcular la impuridad.

In [24]:
class ArbolDecision:
    # :param max_depth: Profundidad máxima del árbol.
    # :param min_split_samples: Número mínimo de muestras necesarias para dividir un nodo.
    # :param criterio: Criterio utilizado para calcular la impureza ("gini" o "entropia").
    def __init__(self, max_depth=1, min_split_samples=2, criterio="gini"):
        self.max_depth = max_depth
        self.min_split_samples = min_split_samples
        self.criterio = criterio
        self.raiz = None

    # Función que calcula el coeficiente de Gini
    def gini_impurity(self, labels):
        # Calculamos la proporción de cada clase
        proportions = labels.value_counts() / len(labels)
        # Calculamos el coeficiente de Gini
        impurity = 1 - sum(proportions**2)
        return impurity
    
    # Función que calcula la entropía
    def entropy_impurity(self, labels):
        # Calculamos la proporción de cada clase
        proportions = labels.value_counts() / len(labels)
        # Calculamos la entropía
        impurity = sum(-proportions * np.log2(proportions))
        return impurity

3. Divida los datos en los conjuntos tradicionales de entrenamiento y prueba, de forma manual, sin utilizar las utilidades de sklearn (puede utilizar índices de Numpy o Pandas)

In [25]:
def dividir_datos(X, y, proporción=0.8):
    indices = np.random.permutation(len(X))
    limite = int(len(X) * proporción)

    X_entrenamiento, y_entrenamiento = X[indices[:limite]], y[indices[:limite]]
    X_prueba, y_prueba = X[indices[limite:]], y[indices[limite:]]

    return X_entrenamiento, X_prueba, y_entrenamiento, y_prueba

## Pruebas

In [26]:
# Carga de datos
data = pd.read_csv("adult-census.csv")
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [27]:
# Agrupación por clase y por adultez
data["umbral"] = data["age"] > 30
data.groupby(["umbral", "class"]).size().unstack("class")

class,<=50K,>50K
umbral,,
False,14800,993
True,22355,10694


In [28]:
# Clasificación de muestras
print(
    "Muestras mal clasificadas con un corte en 25:",
    data.loc[(data["age"]>=25) & (data["class"]==" <=50K"),:].shape[0], "\n",
    "Muestras mal clasificadas con un corte en 50:",
    data.loc[(data["age"]>=50) & (data["class"]==" <=50K"),:].shape[0]
)

Muestras mal clasificadas con un corte en 25: 28816 
 Muestras mal clasificadas con un corte en 50: 7180


In [29]:
# Pruebas gini

# Gini para el dataset completo
print(ArbolDecision().gini_impurity(data["class"]))

# Gini para el dataset dividido por el umbral
print(ArbolDecision().gini_impurity(data[data['age'] > 50]['class']))

0.36405200460016074
0.43390451896643945


In [30]:
# Prueba entropía

# Se calcula la entropía de los adultos mayores a 50 años
ArbolDecision().entropy_impurity(data[data["age"] > 50]["class"])

0.9024238545883967